<a href="https://colab.research.google.com/github/Denzzo-ov/Simulative-Python-HW-cases/blob/main/%D0%90%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D1%81%D1%82%D1%8C_Itresume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **2 кейс**

**Выгрузка активности с ItResume**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [ ]:
!wget https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv

--2026-01-12 19:08:10--  https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv [following]
--2026-01-12 19:08:10--  https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215378 (210K) [text/plain]
Saving to: ‘codesubmit.csv’

codesubmit.csv      100%[===================>] 210.33K  --.-KB/s    in 0.005s  

2026-01-12 19:08:11 (39.7 MB/s) - ‘co

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
import pandas as pd

df = pd.read_csv('codesubmit.csv', sep = ';')
df['created_at'] = pd.to_datetime(df['created_at'])
df

,created_at,user_id,problem_id,is_correct,type
0,2023-04-30 13:47:14.344471,7,870,1.0,submit
1,2023-04-30 13:46:15.949925,7,870,0.0,submit
2,2023-04-30 16:13:26.005286,173,21,1.0,submit
3,2023-04-30 16:13:06.739782,173,21,NaN,run
4,2023-04-30 15:52:00.195532,173,25,1.0,submit
...,...,...,...,...,...
4994,2023-04-30 21:52:00.269123,13493,435,NaN,run
4995,2023-04-30 21:51:01.094234,13493,435,1.0,submit
4996,2023-04-30 21:50:52.059690,13493,435,NaN,run
4997,2023-04-30 21:42:24.323689,13493,1086,NaN,run


### **Решения**

#### **Задача 1**

Ваша задача - выяснить сколько в среднем тратится времени на решение задачи.

**Примечание**: для правильного подсчета - рассчитайте сначала среднее время решения по каждой задаче в отдельности, и только затем находите общее среднее время решения задач.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res`.


**Решение**

Напишите свое решение ниже

In [ ]:
# 1. первая активность
first_activity = df.groupby(['user_id', 'problem_id'])['created_at'].min().rename('first_activity')

# 2. первое верное выполнение
first_correct = df[df['is_correct'] == 1].groupby(['user_id', 'problem_id'])['created_at'].min().rename('first_correct')

# 3. объединяем таблицы и убираем сроки где значения в столбце first_correct пусто
total = pd.concat([first_activity, first_correct], axis=1).dropna(subset=['first_correct']).reset_index()

# 4. оставляем строки в которых время первого ответа не равно времени первой активности
total = total[total['first_activity'] != total['first_correct']].copy()

# 5. считаем время выполнения в секудах
total['solve_time_sec'] = (total['first_correct'] - total['first_activity']).dt.total_seconds()

# 6. считаем среднее время выполнения по одной задаче
time_to_problem = total.groupby('problem_id')['solve_time_sec'].mean()

# 7. Считаем среднее время на решение
res = round(time_to_problem.mean(),2)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res == 611.86
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача - выяснить сколько часов в среднем проводит юзер в день на платформе. Перерывы в активности за день - не учитываем.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res2`.

**Решение**

Напишите свое решение ниже

In [ ]:
from datetime import datetime

df_ = df.copy()
df_["date_cr"] = df_["created_at"].apply(lambda x: x.strftime("%Y-%m-%d"))

df_grouped = df_.groupby(["user_id", "date_cr"]).agg(min_time=("created_at", "min"),
                                                           max_time=("created_at", "max"))
df_grouped['active_time'] = df_grouped['max_time'] - df_grouped['min_time']

res2 = df_grouped['active_time'].mean()
res2 = round(res2.total_seconds() / 3600, 2)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res2 == 1.7
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Теперь давайте посмотрим на активные сеансы. Выясните, сколько задач в среднем решается за один активный сеанс.

**Активный сеанс** - период, когда между любой активностью пользователя разница менее или равна часу, не более

**Важно**: в расчет берем не только успешные попытки решений (`is_correct=1`), а и неуспешные тоже (`is_correct=0`), и тип `run` в том числе.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res3`.

**Решение**

Напишите свое решение ниже

In [ ]:
# сортируем по пользователю и времени
df = df.sort_values(["user_id", "created_at"])

# время предыдущего события по юзеру
df["prev_act"] = df.groupby("user_id")["created_at"].shift(1)

# разница в секундах между текущим и предыдущим событием
df["diff_sec"] = (df["created_at"] - df["prev_act"]).dt.total_seconds()

# отметим начало сеанса как первое событие пользователя или пауза между событиями > 1 часа
df["new_session"] = df["prev_act"].isna() | (df["diff_sec"] > 3600)

# нумеруем сеансы внутри каждого пользователя
df["session_num"] = df.groupby("user_id")["new_session"].cumsum()

# считаем количестов событий за каждый сеанс
tasks_per_session = (df.groupby(["user_id", "session_num"])["problem_id"].nunique())

# среднее количество задач на сеанс
res3 = round(tasks_per_session.mean(), 2)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res3 == 3.14
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 4**

И финальная - найдите самый "популярный" час дня на нашей платформе.

Популярность определяем максимальным количеством уникальных пользователей, совершающих какую-либо активность в этот период

Результат в числовом формате запишите в переменную `res4`.

Например, самым популярным часом стал период с 22 до 23, тогда в переменной `res4` должно лежать **22**. Обозначающее начало этого периода.

**Решение**

Напишите свое решение ниже

In [ ]:
from datetime import datetime

df_4 = df.copy()
# выделяем час из даты
df_4["act_hour"] = df_4["created_at"].apply(lambda x: x.strftime("%H"))

# группируем по часам пользователей
df_gr = df_4.groupby("act_hour")["user_id"].nunique()

# час с нибольшим количеством пользователей
rez = int(df_gr.idxmax())

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res4 == 16
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
